In [83]:
import io
import re


# The dictionary that we use to look up how the words need to be changed to get gender agreement, etc.
linguistic_info = io.open('dela-fr-public.dic','r', encoding='utf-16-le').readlines()
# The vocabulary for the gender, age, nationality, etc. perturbations
vocabulary = io.open('french_vocabulary.csv', 'r+').readlines()
# The sentences we are perturbing to create the evaluation data
sentences = io.open('french_sentences.csv', 'r+').readlines()

# Store the linguistics dictionary
def read_dictionary(ling_info_list):
  word_to_ling = {}
  for item in ling_info_list:
    key = item[0]
    val = item[1]
    if val[0] == '.':
      val = key + val
    word_to_ling[key] = val
  return word_to_ling

def perturb_word(word):
  characteristics = word_characteristics_map[word]
  new_characteristics = re.sub(",m,", ",f,", characteristics)
  new_word = characteristics_word_map[new_characteristics]
  return new_word


In [84]:
ling_info_list = [line.strip('\n').lower().split(',') for line in linguistic_info]
ling_to_word = {entry[1]:entry[0] for entry in ling_info_list}
word_to_ling = read_dictionary(ling_info_list)

word_characteristics_map = {}
characteristics_word_map = {}

for line in vocabulary:
  line = line.strip()
  split_line = line.split(',')
  word = split_line[-1]
  characteristics = ','.join(split_line[:-1])
  word_characteristics_map[word] = characteristics
  characteristics_word_map[characteristics] = word

In [85]:
perturbed_sentence_list = []
for line in sentences:
  words = line.strip().split()
  for word in words:
    word = word.lower()
    if word in word_characteristics_map:
      word = perturb_word(word)
    else:
      details = word_to_ling[word]
      split_gender = details.split(':')
      gender_number = split_gender[1:]
      split_deets = split_gender[0].split('.')
      lemma = split_deets[0]
      pos = split_deets[1]
      #print(lemma)
      #print(pos)
      #print(gender_number)
      if len(gender_number) == 1:
        new_gender_number = re.sub('m', 'f', gender_number[0])
        new_details = lemma + "." + pos + ":" + new_gender_number
        #print(new_details)
        #print(ling_to_word[new_details])
        word = ling_to_word[new_details]
    perturbed_sentence_list += [word]
  print("Result:")
  print(" ".join(perturbed_sentence_list))

Result:
les meufs elles sont toutes nulles
